# BUSINESS UNDERSTANDING

# DATA UNDERSTANDING
### Collecting The Sonic Features
Collecting implicitly labeled songs from playlists such as 'top 100 country songs'. Experiment can be rerun with different genres.

In [1]:
from genres import collect_genre_features
%load_ext autoreload
%autoreload 2

In [2]:
genres = {
    'country': 'https://www.youtube.com/playlist?list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S', 
    'jazz': 'https://www.youtube.com/playlist?list=PL8F6B0753B2CCA128', 
    'hip_hop': 'https://www.youtube.com/playlist?list=PLAPo1R_GVX4IZGbDvUH60bOwIOnZplZzM', 
    'classical': 'https://www.youtube.com/playlist?list=PLRb-5mC4V_Lop8KLXqSqMv4_mqw5M9jjW', 
    'metal': 'https://www.youtube.com/playlist?list=PLfY-m4YMsF-OM1zG80pMguej_Ufm8t0VC', 
    'electronic': 'https://www.youtube.com/playlist?list=PLDDAxmBan0BKeIxuYWjMPBWGXDqNRaW5S'
}

In [3]:
# collect_genre_features(genres)     # Started 8:55 done around 11:28

ERROR: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.
ERROR: This video is no longer available because the YouTube account associated with this video has been terminated.
Sorry about that.
ERROR: This video contains content from Aviator Management GmbH, who has blocked it on copyright grounds.
Sorry about that.
ERROR: This video contains content from UMG, who has blocked it on copyright grounds.
Sorry about that.
ERROR: This video contains content from SME, who has blocked it in your country on copyright grounds.
Sorry about that.
/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
ERROR: The uploader has not made this video available in your country.
Sorry about that.


In [4]:
import pandas as pd

In [5]:
df = pd.read_json('data/genre_features.json', lines=True)

In [15]:
df.groupby('genre').song.nunique()

genre
classical      97
country       100
electronic     99
hip_hop        98
jazz          100
metal         100
Name: song, dtype: int64

# DATA PREPARATION

# MODELING

# EVALUATION

# DEPLOYMENT